In [1]:
import blpapi
from blpapi import Event as EventType
from blpapi import AuthOptions, TlsOptions
from datetime import datetime

In [2]:
# Initialize session options
options = blpapi.SessionOptions()
options.setServerAddress('cloudpoint1.bloomberg.com', 8194, 0)
options.setServerAddress('cloudpoint2.bloomberg.com', 8194, 1)
options.setSessionIdentityOptions(AuthOptions.createWithApp('CHINASILVER:Trading'))

# TLS key files are required for connecting to ZFP endpoint
# They are also generated by BPIPE Admin via EMRS web
options.setTlsOptions(TlsOptions.createFromFiles(r'T:\Intern Folder\External Research\2023 Interns\BPIPE\certificates\65C7B614CC7BF90D81CDE1363A617D85.pk12', '123456', r'T:\Intern Folder\External Research\2023 Interns\BPIPE\certificates\rootCertificate.pk7'))

options.setAutoRestartOnDisconnection(True)
#print(options)

In [3]:
msgList = []

def onEvent(event, session):
    for msg in event:
        if msg.messageType() in ['MarketDepthUpdates','MarketDataEvents']:
            # retrieve correlation ID from each message
            cid = msg.correlationId().value()
            msg = msg.toPy() # dictionary
            if msg['MKTDEPTH_EVENT_TYPE'] == 'MARKET_BY_ORDER' and msg['EID'] == 14112:
                msg['SUBSCRIPTION_TYPE'] = 'TOP'
            elif msg['MKTDEPTH_EVENT_TYPE'] == 'MARKET_BY_ORDER' and msg['EID'] == 53714:
                msg['SUBSCRIPTION_TYPE'] = 'MBO'
            elif msg['MKTDEPTH_EVENT_TYPE'] == 'MARKET_BY_LEVEL' and msg['EID'] == 14112:
                msg['SUBSCRIPTION_TYPE'] = 'MBL'
            msg['cid'] = cid
            msgList.append(msg)
            
        else:
            print(msg)

In [4]:
session = blpapi.Session(options, onEvent)
session.start()

SessionConnectionUp = {
    server = "cloudpoint1.bloomberg.com:8194"
    serverId = "apidm-ob-946-apicszfb"
    encryptionStatus = "Encrypted"
    encryptionCipher = "AES256-GCM-SHA384       TLSv1.2 Kx=RSA      Au=RSA  Enc=AESGCM(256) Mac=AEAD"
    compressionStatus = "Compressed"
}

SessionConnectionUp = {
    server = "cloudpoint2.bloomberg.com:8194"
    serverId = "apidm-rr-913-apicszfb"
    encryptionStatus = "Encrypted"
    encryptionCipher = "AES256-GCM-SHA384       TLSv1.2 Kx=RSA      Au=RSA  Enc=AESGCM(256) Mac=AEAD"
    compressionStatus = "Compressed"
}

CID: {[ valueType=AUTOGEN classId=0 value=1 ]}
RequestId: 7f8c3a05-9999-0a68-0000-34fea8c00500
ServiceOpened = {
    serviceName = "//blp/apiauth"
}

CID: {[ valueType=AUTOGEN classId=0 value=1 ]}
RequestId: 168ca505-999d-0a68-0000-34fea8c00500
TokenGenerationSuccess = {
    token = "-v4;lgHKB5HI7d65C1zuyY9+WjHGQN/G9UUKD0rY12j/jhKl9AQykdqwVBFtF+x0u9HshUyRxdRRWnC0QP8YqannaM879U5BmA+KD3gPA1Kc6Hc=;Pfpx/jbALwgtrEk80jwojTgmoLJjvv

True

CID: {[ valueType=AUTOGEN classId=0 value=1 ]}
AuthorizationSuccess = {
}

SessionStarted = {
    initialEndpoints[] = {
        initialEndpoints = {
            address = "cloudpoint1.bloomberg.com:8194"
        }
        initialEndpoints = {
            address = "cloudpoint2.bloomberg.com:8194"
        }
    }
}



In [5]:
MKTDEPTH_SVC = '//blp/mktdepthdata'
session.openService(MKTDEPTH_SVC)
mktdepth_svc = session.getService(MKTDEPTH_SVC)

CID: {[ valueType=AUTOGEN classId=0 value=8 ]}
RequestId: 5e8ca905-99b0-0a68-0000-34fea8c00500
ServiceOpened = {
    serviceName = "//blp/mktdepthdata"
}



In [6]:
ticker = '3738 HK Equity'

subList = blpapi.SubscriptionList()
subList.add(MKTDEPTH_SVC + '/ticker/'+ str(ticker), options='type=TOP', correlationId=blpapi.CorrelationId(ticker+'.type=TOP'))
subList.add(MKTDEPTH_SVC + '/ticker/'+ str(ticker), options='type=MBO', correlationId=blpapi.CorrelationId(ticker+'.type=MBO')) # Need full book 
subList.add(MKTDEPTH_SVC + '/ticker/'+ str(ticker), options='type=MBL', correlationId=blpapi.CorrelationId(ticker+'.type=MBL'))
#subList.add(f'/ticker/{ticker}', fields=['MKTDATA_EVENT_SUBTYPE,EVT_TRADE_TIME_RT,EVT_TRADE_CONDITION_CODE_RT,EVT_TRADE_PRICE_RT,EVT_TRADE_SIZE_RT'], correlationId=blpapi.CorrelationId(ticker+'.type=QR'))

session.subscribe(subList)

CID: {[ valueType=POINTER classId=0 value=0000024F16A576E0 ]}
RequestId: f68c1405-99b3-0a68-0000-34fea8c00500
SubscriptionStarted = {
    exceptions[] = {
    }
    streamIds[] = {
        "1"
    }
    receivedFrom = {
        address = "cloudpoint1.bloomberg.com:8194"
    }
    reason = "Subscriber made a subscription"
}

CID: {[ valueType=POINTER classId=0 value=0000024F16A575A0 ]}
RequestId: f68c1405-99b3-0a68-0000-34fea8c00500
SubscriptionStarted = {
    exceptions[] = {
    }
    streamIds[] = {
        "2"
    }
    receivedFrom = {
        address = "cloudpoint1.bloomberg.com:8194"
    }
    reason = "Subscriber made a subscription"
}

CID: {[ valueType=POINTER classId=0 value=0000024F16A54E40 ]}
RequestId: f68c1405-99b3-0a68-0000-34fea8c00500
SubscriptionStarted = {
    exceptions[] = {
    }
    streamIds[] = {
        "3"
    }
    receivedFrom = {
        address = "cloudpoint1.bloomberg.com:8194"
    }
    reason = "Subscriber made a subscription"
}

CID: {[ valueType=POIN

In [14]:
msgList

[{'MKTDEPTH_EVENT_TYPE': 'MARKET_BY_LEVEL',
  'MKTDEPTH_EVENT_SUBTYPE': 'TABLE_INITPAINT',
  'ID_BB_SEC_NUM_SRC': 730226313488,
  'FEED_SOURCE': 'HK',
  'EID': 14112,
  'MD_TABLE_CMD_RT': 'REPLACE',
  'MD_BOOK_TYPE': 'MBL-RBP',
  'MBL_WINDOW_SIZE': 10,
  'MD_BOOK_SUBTYPE': 'CLOB',
  'MD_PRICE_DISPLAY_FORMAT_TYPE_RT': 'DECIMAL',
  'MD_PRICE_DISPLAY_MIN_NUM_DECIMALS_RT': 2,
  'MD_PRICE_DISPLAY_MAX_NUM_DECIMALS_RT': 3,
  'MD_CURRENCY_RT': 'HKD',
  'MD_QUOTED_CURRENCY_REALTIME': 'HKD',
  'MD_ROUND_LOT_SIZE_RT': 1000,
  'SUBSCRIPTION_TYPE': 'MBL'},
 {'MKTDEPTH_EVENT_TYPE': 'MARKET_BY_ORDER',
  'MKTDEPTH_EVENT_SUBTYPE': 'TABLE_INITPAINT',
  'ID_BB_SEC_NUM_SRC': 730226313488,
  'FEED_SOURCE': 'HK',
  'EID': 14112,
  'MD_TABLE_CMD_RT': 'REPLACE',
  'MD_BOOK_TYPE': 'TOP-RBP',
  'MBO_WINDOW_SIZE': 40,
  'MD_BOOK_SUBTYPE': 'TOPBROKER',
  'MD_PRICE_DISPLAY_FORMAT_TYPE_RT': 'DECIMAL',
  'MD_PRICE_DISPLAY_MIN_NUM_DECIMALS_RT': 2,
  'MD_PRICE_DISPLAY_MAX_NUM_DECIMALS_RT': 3,
  'MD_CURRENCY_RT': 'HKD'

In [15]:
print(len(msgList))

2793


In [14]:
msg = msgList[2] # if 'MKTDEPTH_EVENT_TYPE' == 'MARKET_BY_ORDER'; 'MKTDEPTH_EVENT_SUBTYPE' == 'TABLE_INITPAINT'
for x in msg['MBO_TABLE_ASK']:
    print(f"Ask {x['MBO_ASK_POSITION_RT']}: Broker {x['MBO_ASK_BROKER_RT']} offering at {x['MBO_ASK_RT']} on {x['MBO_TIME_RT'].astimezone()}")
    
for x in msg['MBO_TABLE_BID']:
    print(f"Bid {x['MBO_BID_POSITION_RT']}: Broker {x['MBO_BID_BROKER_RT']} bidding at {x['MBO_BID_RT']} on {x['MBO_TIME_RT'].astimezone()}")

Ask 1: Broker 1836 offering at 2.40000057220459 on 2023-06-12 10:16:58+08:00
Ask 2: Broker 5998 offering at 2.40000057220459 on 2023-06-12 10:16:58+08:00
Ask 3: Broker 3436 offering at 2.40000057220459 on 2023-06-12 10:19:12+08:00
Ask 4: Broker 5346 offering at 2.40000057220459 on 2023-06-12 10:19:39+08:00
Ask 5: Broker 5999 offering at 2.4099998474121094 on 2023-06-12 10:22:15+08:00
Ask 6: Broker 6387 offering at 2.4099998474121094 on 2023-06-12 10:22:15+08:00
Ask 7: Broker 5998 offering at 2.4099998474121094 on 2023-06-12 10:22:15+08:00
Ask 8: Broker 6387 offering at 2.4099998474121094 on 2023-06-12 10:22:15+08:00
Ask 9: Broker 6387 offering at 2.4099998474121094 on 2023-06-12 10:18:22+08:00
Ask 10: Broker 6387 offering at 2.4099998474121094 on 2023-06-12 10:19:39+08:00
Ask 11: Broker 6387 offering at 2.4099998474121094 on 2023-06-12 10:20:14+08:00
Ask 12: Broker 6387 offering at 2.4099998474121094 on 2023-06-12 10:20:45+08:00
Ask 13: Broker 5999 offering at 2.4099998474121094 on 202

In [29]:
msgList[1]

{'MKTDEPTH_EVENT_TYPE': 'MARKET_BY_LEVEL',
 'MKTDEPTH_EVENT_SUBTYPE': 'TABLE_INITPAINT',
 'ID_BB_SEC_NUM_SRC': 730145003912,
 'FEED_SOURCE': 'HK',
 'EID': 14112,
 'MD_TABLE_CMD_RT': 'REPLACE',
 'MD_BOOK_TYPE': 'MBL-RBP',
 'MBL_WINDOW_SIZE': 10,
 'MD_BOOK_SUBTYPE': 'CLOB',
 'MD_PRICE_DISPLAY_FORMAT_TYPE_RT': 'DECIMAL',
 'MD_PRICE_DISPLAY_MIN_NUM_DECIMALS_RT': 2,
 'MD_PRICE_DISPLAY_MAX_NUM_DECIMALS_RT': 3,
 'MD_CURRENCY_RT': 'HKD',
 'MD_QUOTED_CURRENCY_REALTIME': 'HKD',
 'MD_ROUND_LOT_SIZE_RT': 2000,
 'MBL_TABLE_ASK': [{'MBL_ASK_POSITION_RT': 1,
   'MBL_ASK_RT': 0.9200000166893005,
   'MBL_ASK_COND_CODE_RT': '',
   'MBL_ASK_NUM_ORDERS_RT': 1,
   'MBL_ASK_SIZE_RT': 12000,
   'MBL_TIME_RT': datetime.datetime(2023, 6, 9, 1, 30, 18, 21000, tzinfo=<blpapi.datetime.FixedOffset object at 0x000001E4ABAC6CB0>),
   'MD_TABLE_CMD_RT': 'REPLACE'},
  {'MBL_ASK_POSITION_RT': 2,
   'MBL_ASK_RT': 1.0,
   'MBL_ASK_COND_CODE_RT': '',
   'MBL_ASK_NUM_ORDERS_RT': 1,
   'MBL_ASK_SIZE_RT': 10000,
   'MBL_TIME

In [ ]:
datetime.microsecond

In [27]:
x['MBO_TIME_RT']

datetime.datetime(2023, 6, 9, 1, 32, 59, tzinfo=<blpapi.datetime.FixedOffset object at 0x000001E4ABAC7520>)

In [28]:
# DEMO only: cancel here once enough messages are received
session.cancel(cid)

CID: {[ valueType=POINTER classId=0 value=00000202E734CFD0 ]}
SubscriptionTerminated = {
    reason = {
        source = "SubscriptionManager"
        category = "CANCELLED"
        errorCode = 0
        description = "Subscription cancelled"
    }
}



In [6]:
session.stop()

True

In [16]:
import pickle

with open('3738_HK_msgList_0616.pickle','wb') as f:
    pickle.dump(msgList, f)

#msgList

In [1]:
import pickle

with open('3738_msgList.pickle','rb') as f:
    msgList = pickle.load(f)

In [9]:
msgList

[{'MKTDEPTH_EVENT_TYPE': 'MARKET_BY_ORDER',
  'MKTDEPTH_EVENT_SUBTYPE': 'TABLE_INITPAINT',
  'ID_BB_SEC_NUM_SRC': 730145003912,
  'FEED_SOURCE': 'HK',
  'EID': 53714,
  'MD_TABLE_CMD_RT': 'ADD',
  'MD_BOOK_TYPE': 'MBO-AMD',
  'MBO_WINDOW_SIZE': 1000,
  'MBO_SEQNUM_RT': 7,
  'MD_BOOK_SUBTYPE': 'CLOB',
  'MD_PRICE_DISPLAY_FORMAT_TYPE_RT': 'DECIMAL',
  'MD_PRICE_DISPLAY_MIN_NUM_DECIMALS_RT': 2,
  'MD_PRICE_DISPLAY_MAX_NUM_DECIMALS_RT': 3,
  'MD_CURRENCY_RT': 'HKD',
  'MD_QUOTED_CURRENCY_REALTIME': 'HKD',
  'MD_ROUND_LOT_SIZE_RT': 2000,
  'MBO_TABLE_ASK': [{'MBO_ASK_POSITION_RT': 1,
    'MBO_ASK_RT': 0.8999999761581421,
    'MBO_ASK_COND_CODE_RT': '',
    'MBO_ORDER_ID_RT': '9121025',
    'MBO_ASK_SIZE_RT': 2000,
    'MBO_TIME_RT': datetime.datetime(2023, 6, 14, 1, 20, 20, 635000, tzinfo=<blpapi.datetime.FixedOffset object at 0x0000013A58307A30>),
    'MD_TABLE_CMD_RT': 'ADD'},
   {'MBO_ASK_POSITION_RT': 2,
    'MBO_ASK_RT': 1.0,
    'MBO_ASK_COND_CODE_RT': '',
    'MBO_ORDER_ID_RT': '2868

In [11]:
msgList[-1]

{'MKTDEPTH_EVENT_TYPE': 'MARKET_BY_ORDER',
 'MKTDEPTH_EVENT_SUBTYPE': 'BID',
 'EID': 14112,
 'MD_TABLE_CMD_RT': 'REPLACE',
 'MBO_TIME_RT': datetime.datetime(2023, 6, 15, 1, 36, 50, tzinfo=<blpapi.datetime.FixedOffset object at 0x0000024DB94777F0>),
 'MBO_BID_POSITION_RT': 2,
 'MBO_BID_RT': 0.800000011920929,
 'MBO_BID_BROKER_RT': ' 298',
 'MBO_BID_COND_CODE_RT': '',
 'MBO_BID_SIZE_RT': 0,
 'SUBSCRIPTION_TYPE': 'TOP'}

In [17]:
from blp import blp
from datetime import timedelta
bquery = blp.BlpQuery().start()
df = bquery.bdit(security='3738 HK Equity', event_types = ['TRADE'], start_datetime='2023-06-16T01:00:00', end_datetime = '2023-06-16T09:00:00', options=[('includeConditionCodes',True), ('includeNonPlottableEvents', True)])
df['time']+=timedelta(hours=8)

In [18]:
import pickle
with open('3738_HK_trades_0616.pickle','wb') as f:
    pickle.dump(df, f)